In [1]:
## Loading the Training Data

import csv
import numpy as np


def trainLoading(file_name):
    
    file_contents = []
    labels = []
    datas = []
    
    
    with open(file_name) as file :
        
        lines = csv.reader(file)
        
        for line in lines:
            
            file_contents.append(line)
    
    file_contents.remove(file_contents[0])
    
    for line in file_contents:
        
        labels.append(labelArray(line[0]))
        datas.append(list(map(binarizeChar,line[1:])))
    
    labels = np.array(labels)
    datas = np.array(datas)
    
    return (labels,datas)
    
    
def testLoading(file_name):
    
    file_contents = []
    datas = []
    with open(file_name) as file :
        
        lines = csv.reader(file)
        
        for line in lines:
            
            file_contents.append(line)
    
    file_contents.remove(file_contents[0])
    
    for line in file_contents:

        datas.append(list(map(binarizeChar,line)))
        
    datas = np.array(datas)
    
    return datas


def binarizeChar(c):
    if c == '0':
        return 0
    else :
        return 1

def labelArray(l):
    arr = [0]*10
    arr[int(l)] = 1
    return arr
    
    
    
if __name__ == '__main__':
    
    (l,d) = trainLoading('train.csv')
    test_data = testLoading('test.csv')
    

In [25]:
import tensorflow as tf
import random 

## Delete the current graph
tf.reset_default_graph()


def CNN(x):
    
    input_layer = tf.reshape(x,[-1,28,28,1])
    
    conv_1 = tf.layers.conv2d(inputs=input_layer,filters=300,kernel_size=[3,3],padding='same',name='Conv_Layer_1',activation=tf.nn.relu)
    
    max_pool_1 = tf.layers.max_pooling2d(inputs=conv_1,pool_size=[2,2],strides=2,name='Max_Pool_1')
    
    flat = tf.reshape(max_pool_1,[-1,14*14*300])
    
    dense_layer = tf.layers.dense(inputs=flat,units=128,activation=tf.nn.relu)
    
    output_layer = tf.layers.dense(inputs=dense_layer,units=10)
    
    
    
    
    return output_layer
    

x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])


logits = CNN(x)

y = tf.nn.softmax(logits)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y_)

#optimizer = tf.train.AdamOptimizer(learning_rate=0.005)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)

train_step = optimizer.minimize(cross_entropy)

predict = tf.argmax(y,axis=1)

estimate = tf.equal(predict,tf.argmax(y_,axis=1))

accuarcy = tf.reduce_mean(tf.cast(estimate,tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    epoch = 6000
    batch_size = 50
    s = 0
    
    for i in range(epoch):
        
        sess.run(train_step,feed_dict={x:d[s:s+batch_size],y_:l[s:s+batch_size]})
        
        s = s + batch_size
        
        if i % (epoch/20) == 0:
        
            seed = random.randint(0,41000)

            acc = sess.run(accuarcy,feed_dict={x:d[seed:seed+1000],y_:l[seed:seed+1000]})

            print(acc)
            
            if(acc>0.985):
                
                print('I got a 98.5% model !!')
                
                break
                
    s = 0
    submit_prediction = np.array([])
    
    for j in range(28):
        
        submit_prediction_part = sess.run(tf.cast(predict,tf.int32),feed_dict={x:test_data[s:s+1000]})
        submit_prediction = np.concatenate((submit_prediction,submit_prediction_part))
        
        s = s + 1000
        
    submit_prediction = np.array(list(map(int,submit_prediction)))

0.136
0.955
0.964
0.986
I got a 98.5% model !!


array([2, 0, 9, ..., 3, 9, 2])

[0, 0, [1], 6]